## OP.GG 게임 전적 데이터 수집하기 [칼바람 나락]

 - URL : https://op.gg
 
### < 크롤링 구현에서의 사용자의 입력 내용 >
- 닉네임 입력 
- 수집할 데이터 개수 입력 : 수집할 데이터는 1게임 당 한 행에 해당

### < 수집 데이터 목록 >
1. 승패 여부
2. 시간
3. 챔피언 처치
4. 골드 획득량
5. 챔피언에게 가한 피해량
6. 받은 피해량
7. CS
8. 승리팀의 First Blood 여부
9. 승리팀의 First Tower Destroyed 여부

### < 크롤링 과정 >
※ op.gg 사이트 들어온 후 과정
1. <b>닉네임 입력</b>
  - 1.1 닉네임 검색 창에 닉네임 입력
  - 1.2 Enter Or 검색 버튼 누르기
<br><br>
2. <b>최신 데이터 값까지 받기 위한 전적 갱신</b>
  - 2.1 전적 갱신 버튼 누르기
<br><br>
3. <b>무작위 총력전 전적만 나오도록하기</b>
  - 큐타입 -> 무작위 총력전 선택
<br><br>
4. <b>게임 리스트 더보기 버튼 누르기</b>
  - 요청당 20개의 게임 데이터만 불러와지므로,
  - 요청 데이터가 20판 수 단위로 더보기 버튼 클릭
<br><br>
5. <b>게임 상세 정보 버튼 누르기</b>
  - 게임 판당 게임 상세 정보 버튼 누르기
  - 입력했던 count_txt 만큼 반복
<br><br>
6. <b>팀분석 버튼 누르기</b>
  - 한 게임당 게임 상세 정보를 통해 나온 하위 영역의 팀분석 버튼 누르기
<br><br>
7. <b>해당 사이트의 HTML 분석 및 데이터 필요 데이터 추출</b>
  - 요청으로 얻어온 모든 HTML을 수집하여 필요한 데이터를 분석 및 추출
<br><br>
8. <b>추출한 데이터를 DataFrame으로 변환</b>
  - HTML에서 분석 및 추출한 데이터를 DataFrame으로 변환
<br><br>
9. <b>원본 데이터를 놔두고 사본 데이터를 생성하여 데이터 전처리</b>
  - 문자값으로 되어있는 데이터를 분석하기 쉽도록 숫자값으로 변환
  - 문자열로 나눠지는 것은 기준을 정하여 조건에 맞게 변환
<br><br>
10. <b>정리한 데이터를 csv 파일로 출력</b>
  - 원본 데이터와, 변환된 데이터를 CSV 파일로 출력

## Step 1. 필요한 모듈과 함수 로딩하기

In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import os
import pandas as pd
import sys
import math
import re

## Step 2. 크롤링에 필요한 정보 입력 및 가공

### 2.1 필요한 정보(닉네임, 크롤링 개수) 입력 받기

In [8]:
url = "https://op.gg"

id_txt = input("1. 닉네임 입력 : ")

if id_txt == "" : 
    id_txt = "으흐이"
    
try:
    count_txt = int(input("2. 크롤링 개수 : "))
except:
    count_txt = 20
    
print("=" * 30)
print("입력 닉네임 :", id_txt)
print("입력 개수 :", count_txt)
print("=" * 30)

1. 닉네임 입력 : 
2. 크롤링 개수 : 
입력 닉네임 : 으흐이
입력 개수 : 20


### 2.2 크롤링 할 데이터에 맞게 카운트  계산
 - request당 20개의 데이터가 불러와짐
 - reuqest는 최하단 더보기를 클릭
 - real_cnt 만큼 더보기를 클릭


In [52]:
real_cnt = math.ceil(count_txt / 20) - 1

print(real_cnt)

4


## Step 3. 데이터 저장 경로 및 구글 드라이버 경로 설정

### 3.1 데이터 저장 경로 설정
 -  현재 작업 경로에 "OPGG_Crawling_save_data" 폴더안에 데이터를 저장할 폴더를 생성<br>
    <b>※ 현재 작업 경로에 OPGG_Crawling_save_data 폴더가 존재해야 함.</b>
 -  데이터가 저장될 폴더명 : 닉네임_현재년월일시분초
 -  데이터가 저장될 파일명 : 닉네임_수집개수

In [9]:
# 폴더 만들어 주기
save_path = 'OPGG_Crawling_save_data'
now = time.localtime()

data_hms = '%04d-%02d-%02d-%02d-%02d-%02d' %(now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
dir_name = save_path + "\\" + id_txt + '_' + data_hms
'OPGG_Crawling_save_data\\으흐이_2021-07-11-01-58-42'
os.mkdir(dir_name)

In [54]:
# 파일명 설정
save_name = id_txt + "cnt(" + str(count_txt) + ")"

In [55]:
# 생성되었는지 확인
os.listdir(save_path)

['0. ALL_Data',
 '154kv덕림_2021-07-11-16-00-00',
 'MTL에드먼_2021-07-20-14-36-33',
 '경상도블라디_2021-07-20-12-40-38',
 '당크매_2021-07-12-21-58-40',
 '로니팡_2021-07-20-10-02-20',
 '리리찍_2021-07-20-12-43-36',
 '백골부대특전사_2021-07-20-10-45-31',
 '으흐이_2021-07-20-10-33-24',
 '으흐이_2021-07-20-11-12-06']

### 3.2 크롬 드라이버 경로 설정
- 해당 작업경로에 "0.Chrome_Driver" 폴더 안에 존재하는 크롬 드라이버를 사용

In [26]:
driva_path = "0. Chrome_Driver\\chromedriver.exe"

## Step 4. 크롬 드라이버를 사용해서 웹 브라우저 실행 후 OP.GG 접속

In [8]:
driver = webdriver.Chrome(driva_path)

driver.get(url)
time.sleep(2)

## Step 5. OP.GG 닉네임 검색창에 입력받은 닉네임 넣고 검색 실행

In [56]:
element = driver.find_element_by_name("userName")
element.send_keys(id_txt)
element.submit() # Enter
time.sleep(1)

## Step 6. 전적 갱신 버튼 클릭

In [57]:
driver.find_element_by_id("SummonerRefreshButton").click()
time.sleep(1)

## Step 7. 무작위 총력전 전적만 나오도록 선택

In [59]:
driver.find_element_by_xpath('//*[@id="right_gametype_queuetype"]/span/select/option[3]').click()
time.sleep(1)

## Step 8. 전적 더보기 버튼 클릭
 - 한 Game List에 제공되는 게임 수 : 20
 - 크롤링할 데이터가 20개 단위 마다 더보기 버튼을 클릭
 - 버튼을 클릭할 때마다 Xpath의 div[] 값이 1씩 증가
 - 게임 데이터가 모두 출력되어 더보기 버튼이 활성화되지 않을 시 예외처리

In [60]:
for n in range(real_cnt):
    try:
        driver.find_element_by_xpath('//*[@id="SummonerLayoutContent"]/div[2]/div[2]/div/div[2]/div[' + str((4 + n)) + ']/a').click()
    except:
        print("더 이상 전적 정보가 없습니다.")
        break
    time.sleep(1)

## Step 9. 팀분석 탭 활성화
 - 게임 정보에 필요한 팀분석 정보는 버튼을 클릭하여 탭을 활성화 시켰을 때, 즉<br>
   사용자가 요청할 때 정보를 불러오므로 판수마다 반복하여 클릭을 해주어야 함
 - 게임 한판의 정보가 아닌 내가 카운트한 판 수의 정보를 불러와야 함으로 Xpath를 통해 반복시켜주도록 함

In [61]:
# 게임 정보 더보기란에 있는 팀 분석 탭을 클릭하기 위해, 더보기 버튼 클릭

# div xpath 기본값이 3
# 20개 마다 1씩 상승
div_cnt = 3

# div_cnt가 1 상승 할 때 
# 1로 초기화
div_div_cnt = 1

for _ in range(count_txt):
    
    if div_div_cnt % 21 == 0:
        div_cnt += 1
        div_div_cnt = 1

    driver.find_element_by_xpath(f'//*[@id="SummonerLayoutContent"]/div[2]/div[2]/div/div[2]/div[{div_cnt}]/div[{div_div_cnt}]/div/div[1]/div[7]/div/div').click()
    
    div_div_cnt += 1
    time.sleep(0.5)

In [62]:
# 팀 분석 xpath 값을 리스트로 받아와 반복    
list_ = driver.find_elements_by_xpath('//*[@id="right_match_team"]')
for i in range(count_txt):
    list_[i].click()
    time.sleep(0.7)

## Step 10. 필요한 정보 태그를 리스트에 할당

- div.GameItemList : 20개의 게임에 대한 데이터 그룹
    - div.GameItemWrap : 게임 한판에 대한 데이터

In [63]:
# op.gg 의 전체 html 소스를 text 형태로 담아줌
full_html = driver.page_source

# text 형태의 html을 parsing??? 모르겠다
soup = BeautifulSoup(full_html, 'html.parser')

# 필요한 데이터만 추출
# 게임 한판에 대한 모든 데이터를 추출
content_list = soup.find_all('div', class_='GameItemWrap')[:count_txt]
print("총 게임 판 수 :", len(content_list))

총 게임 판 수 : 100


## Step 11. 게임 데이터에서 필요한 정보 추출
- 승패 여부 : div.GameResult
- 팀 : th.HeaderCell
- 게임 시간 : div.GameLength
<hr>
<b>해당 데이터를 수집 중 Error 발생 할 수도 있으므로, 예외처리 구문 생성</b>
1. 팀분석 탭이 활성화되지 않아서 데이터를 못가지고 올 경우<br>
   해당 판 데이터만 보류 후 다음 데이터로 진행

- 챔피언 처치
  - 승리팀 챔피언 처치 : div.Value Result-WIN[0]
  - 패배팀 챔피언 처리 : div.Value Result-LOSE[0]
- 골드 획득량
  - 승리팀 챔피언 처치 : div.Value Result-WIN[1]
  - 패배팀 챔피언 처리 : div.Value Result-LOSE[1]
- 챔피언에게 가한 피해량
  - 승리팀 챔피언에게 가한 피해량 : div.Value Result-WIN[2]
  - 패배팀 챔피언에게 가한 피해량 : div.Value Result-LOSE[2]
- 받은 피해량
  - 승리팀 받은 피해량 : div.Value Result-WIN[4]
  - 패배팀 받은 피해량 : div.Value Result-LOSE[4]
- CS
  - 승리팀 CS : div.Value Result-WIN[5]
  - 패배팀 CS : div.Value Result-LOSE[5]
<hr>
- 가져오는 기준이 애매하여 일단 보류
  - 승리팀의 First Blood 여부
  - 승리팀의 First Tower Destroyed 여부

In [63]:
GameResult = []
Team = []
GameLength = []
Win_Champion_kill = []
Lose_Champion_kill = []
Win_Money = []
Lose_Money = []
Win_in_Attack_num = []
Lose_in_Attack_num = []
Win_out_Attack_num = []
Lose_out_Attack_num = []
Win_CS_Count = []
Lose_CS_Count = []

for i in range(len(content_list)):
    
    try:
        Winner_list = content_list[i].find_all("div", class_ = 'Value Result-WIN')
        Loser_list = content_list[i].find_all("div", class_ = 'Value Result-LOSE')
        Win_Champion_kill_val = Winner_list[0].text
        Lose_Champion_kill_val = Loser_list[0].text
    except:
        print("☆★☆★☆★☆★해당 데이터를 가지고 오지 못했습니다★☆★☆★☆★☆")
        continue
        
    GameResult_val = re.search(r'\w\w', content_list[i].find("div", class_ = "GameResult").text).group()
    GameResult.append(GameResult_val)
    
    Team_val = re.search(r'\w\w\w', content_list[i].find("th", class_="HeaderCell").text).group()
    Team.append(Team_val)
    
    GameLength_val = content_list[i].find("div", class_="GameLength").text
    GameLength.append(GameLength_val)
    
    Win_Champion_kill.append(Win_Champion_kill_val)
    Lose_Champion_kill.append(Lose_Champion_kill_val)
    
    Win_Money_val = Winner_list[1].text
    Win_Money.append(Win_Money_val)
    Lose_Money_val = Loser_list[1].text
    Lose_Money.append(Lose_Money_val)
    
    Win_in_Attack_num_val = Winner_list[2].text
    Win_in_Attack_num.append(Win_in_Attack_num_val)
    Lose_in_Attack_num_val = Loser_list[2].text
    Lose_in_Attack_num.append(Lose_in_Attack_num_val)
    
    Win_out_Attack_num_val = Winner_list[4].text
    Win_out_Attack_num.append(Win_out_Attack_num_val)
    Lose_out_Attack_num_val = Loser_list[4].text
    Lose_out_Attack_num.append(Lose_out_Attack_num_val)
    
    Win_CS_Count_val = Winner_list[5].text
    Win_CS_Count.append(Win_CS_Count_val)
    Lose_CS_Count_val = Loser_list[5].text
    Lose_CS_Count.append(Lose_CS_Count_val)

    print(f"{i+1}판 게임 정보")
    print("=" * 50)
    print(f"승리 여부 \t\t: {GameResult_val}")
    print(f"팀 \t\t\t: {Team_val}")
    print(f"게임 시간 \t\t: {GameLength_val}")
    print(f"승리팀 챔피언 킬 \t\t: {Win_Champion_kill_val}")
    print(f"패배팀 챔피언 킬 \t\t: {Lose_Champion_kill_val}")
    print(f"승리팀 골드량 \t\t: {Win_Money_val}")
    print(f"패배팀 골드량 \t\t: {Lose_Money_val}")
    print(f"승리팀 딜량 \t\t: {Win_in_Attack_num_val}")
    print(f"패배팀 딜량 \t\t: {Lose_in_Attack_num_val}")
    print(f"승리팀 받은 딜량 \t\t: {Win_out_Attack_num_val}")
    print(f"패배팀 받은 딜량 \t\t: {Lose_out_Attack_num_val}")
    print(f"승리팀 CS 개수 \t\t: {Win_CS_Count_val}")
    print(f"패배팀 CS 개수 \t\t: {Lose_CS_Count_val}")
    print("=" * 50, end = "\n\n")
    

1판 게임 정보
승리 여부 		: 승리
팀 			: 레드팀
게임 시간 		: 19분 17초
승리팀 챔피언 킬 		: 51
패배팀 챔피언 킬 		: 51
승리팀 골드량 		: 73,527
패배팀 골드량 		: 69,816
승리팀 딜량 		: 100,149
패배팀 딜량 		: 120,379
승리팀 받은 딜량 		: 132,125
패배팀 받은 딜량 		: 108,126
승리팀 CS 개수 		: 221
패배팀 CS 개수 		: 209

2판 게임 정보
승리 여부 		: 패배
팀 			: 블루팀
게임 시간 		: 20분 18초
승리팀 챔피언 킬 		: 25
패배팀 챔피언 킬 		: 21
승리팀 골드량 		: 63,076
패배팀 골드량 		: 60,438
승리팀 딜량 		: 100,951
패배팀 딜량 		: 67,485
승리팀 받은 딜량 		: 71,404
패배팀 받은 딜량 		: 105,481
승리팀 CS 개수 		: 239
패배팀 CS 개수 		: 220

3판 게임 정보
승리 여부 		: 패배
팀 			: 레드팀
게임 시간 		: 9분 5초
승리팀 챔피언 킬 		: 34
패배팀 챔피언 킬 		: 22
승리팀 골드량 		: 36,879
패배팀 골드량 		: 33,504
승리팀 딜량 		: 51,300
패배팀 딜량 		: 42,176
승리팀 받은 딜량 		: 50,545
패배팀 받은 딜량 		: 56,216
승리팀 CS 개수 		: 99
패배팀 CS 개수 		: 73

4판 게임 정보
승리 여부 		: 패배
팀 			: 레드팀
게임 시간 		: 18분 58초
승리팀 챔피언 킬 		: 53
패배팀 챔피언 킬 		: 51
승리팀 골드량 		: 68,069
패배팀 골드량 		: 66,054
승리팀 딜량 		: 144,150
패배팀 딜량 		: 138,472
승리팀 받은 딜량 		: 155,341
패배팀 받은 딜량 		: 151,370
승리팀 CS 개수 		: 209
패배팀 CS 개수 		: 206

5판 게임 정보
승리 여부 		: 승리
팀 			: 블루팀
게임 시간 		:

37판 게임 정보
승리 여부 		: 패배
팀 			: 레드팀
게임 시간 		: 13분 49초
승리팀 챔피언 킬 		: 37
패배팀 챔피언 킬 		: 28
승리팀 골드량 		: 52,108
패배팀 골드량 		: 46,402
승리팀 딜량 		: 77,006
패배팀 딜량 		: 64,782
승리팀 받은 딜량 		: 72,533
패배팀 받은 딜량 		: 81,878
승리팀 CS 개수 		: 158
패배팀 CS 개수 		: 127

☆★☆★☆★☆★해당 데이터를 가지고 오지 못했습니다★☆★☆★☆★☆
39판 게임 정보
승리 여부 		: 승리
팀 			: 블루팀
게임 시간 		: 17분 53초
승리팀 챔피언 킬 		: 54
패배팀 챔피언 킬 		: 43
승리팀 골드량 		: 65,497
패배팀 골드량 		: 60,972
승리팀 딜량 		: 158,518
패배팀 딜량 		: 115,103
승리팀 받은 딜량 		: 121,344
패배팀 받은 딜량 		: 170,386
승리팀 CS 개수 		: 213
패배팀 CS 개수 		: 187

40판 게임 정보
승리 여부 		: 승리
팀 			: 블루팀
게임 시간 		: 19분 37초
승리팀 챔피언 킬 		: 48
패배팀 챔피언 킬 		: 47
승리팀 골드량 		: 68,376
패배팀 골드량 		: 66,376
승리팀 딜량 		: 147,994
패배팀 딜량 		: 104,603
승리팀 받은 딜량 		: 110,857
패배팀 받은 딜량 		: 155,560
승리팀 CS 개수 		: 236
패배팀 CS 개수 		: 233

41판 게임 정보
승리 여부 		: 승리
팀 			: 블루팀
게임 시간 		: 16분 27초
승리팀 챔피언 킬 		: 49
패배팀 챔피언 킬 		: 34
승리팀 골드량 		: 61,882
패배팀 골드량 		: 55,591
승리팀 딜량 		: 120,681
패배팀 딜량 		: 87,786
승리팀 받은 딜량 		: 96,472
패배팀 받은 딜량 		: 132,490
승리팀 CS 개수 		: 205
패배팀 CS 개수 		: 18

73판 게임 정보
승리 여부 		: 승리
팀 			: 블루팀
게임 시간 		: 18분 36초
승리팀 챔피언 킬 		: 44
패배팀 챔피언 킬 		: 33
승리팀 골드량 		: 65,554
패배팀 골드량 		: 60,127
승리팀 딜량 		: 121,322
패배팀 딜량 		: 130,654
승리팀 받은 딜량 		: 139,932
패배팀 받은 딜량 		: 124,966
승리팀 CS 개수 		: 222
패배팀 CS 개수 		: 219

74판 게임 정보
승리 여부 		: 승리
팀 			: 레드팀
게임 시간 		: 21분 45초
승리팀 챔피언 킬 		: 56
패배팀 챔피언 킬 		: 41
승리팀 골드량 		: 74,358
패배팀 골드량 		: 70,544
승리팀 딜량 		: 131,962
패배팀 딜량 		: 122,626
승리팀 받은 딜량 		: 130,668
패배팀 받은 딜량 		: 136,354
승리팀 CS 개수 		: 226
패배팀 CS 개수 		: 296

75판 게임 정보
승리 여부 		: 패배
팀 			: 레드팀
게임 시간 		: 19분 57초
승리팀 챔피언 킬 		: 53
패배팀 챔피언 킬 		: 42
승리팀 골드량 		: 72,549
패배팀 골드량 		: 66,356
승리팀 딜량 		: 125,816
패배팀 딜량 		: 96,128
승리팀 받은 딜량 		: 103,285
패배팀 받은 딜량 		: 139,059
승리팀 CS 개수 		: 237
패배팀 CS 개수 		: 212

☆★☆★☆★☆★해당 데이터를 가지고 오지 못했습니다★☆★☆★☆★☆
77판 게임 정보
승리 여부 		: 패배
팀 			: 레드팀
게임 시간 		: 10분 1초
승리팀 챔피언 킬 		: 39
패배팀 챔피언 킬 		: 22
승리팀 골드량 		: 43,350
패배팀 골드량 		: 34,686
승리팀 딜량 		: 63,219
패배팀 딜량 		: 40,848
승리팀 받은 딜량 		: 53,447
패배팀 받은 딜량 		: 69,579
승리팀 CS 개수 		: 126
패배팀 CS 개수 		: 83

## Step 12. 저장된 데이터 DataFrame으로  만들기

In [64]:
OPGG_df = pd.DataFrame()
OPGG_df['GameResult'] = GameResult
OPGG_df['Team'] = Team
OPGG_df['GameLength'] = GameLength
OPGG_df['Win_Champion_kill'] = Win_Champion_kill
OPGG_df['Lose_Champion_kill'] = Lose_Champion_kill
OPGG_df['Win_Money'] = Win_Money
OPGG_df['Lose_Money'] = Lose_Money
OPGG_df['Win_in_Attack_num'] = Win_in_Attack_num
OPGG_df['Lose_in_Attack_num'] = Lose_in_Attack_num
OPGG_df['Win_out_Attack_num'] = Win_out_Attack_num
OPGG_df['Lose_out_Attack_num'] = Lose_out_Attack_num
OPGG_df['Win_CS_Count'] = Win_CS_Count
OPGG_df['Lose_CS_Count'] = Lose_CS_Count

OPGG_df

,GameResult,Team,GameLength,Win_Champion_kill,Lose_Champion_kill,Win_Money,Lose_Money,Win_in_Attack_num,Lose_in_Attack_num,Win_out_Attack_num,Lose_out_Attack_num,Win_CS_Count,Lose_CS_Count
0,승리,레드팀,19분 17초,51,51,"73,527","69,816","100,149","120,379","132,125","108,126",221,209
1,패배,블루팀,20분 18초,25,21,"63,076","60,438","100,951","67,485","71,404","105,481",239,220
2,패배,레드팀,9분 5초,34,22,"36,879","33,504","51,300","42,176","50,545","56,216",99,73
3,패배,레드팀,18분 58초,53,51,"68,069","66,054","144,150","138,472","155,341","151,370",209,206
4,승리,블루팀,16분 14초,31,23,"55,895","51,306","70,761","69,983","74,060","75,905",197,192
...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,패배,레드팀,16분 16초,46,35,"60,715","54,400","98,304","93,747","107,576","104,652",195,164
93,승리,블루팀,19분 22초,42,46,"66,111","64,496","103,216","112,600","122,896","107,625",235,235
94,패배,레드팀,19분 49초,45,44,"68,619","67,188","119,711","111,846","119,952","124,372",234,245
95,승리,블루팀,20분 10초,39,42,"67,918","65,295","89,534","118,738","127,415","91,888",250,235


## Step 13. DataFrame 값 바꿔주기
- 변환된 데이터를 담을 DataFrame을 생성하여 조건에 맞도록 함수를 만들어<br>
  모든 열에 적용시켜서 담아줌

### 13.1 조건에 맞게 바꿔주도록 함수 생성
- 1. GameResult : 승리(1), 패배(0) 값 변경
- 2. Team : 레드팀(1), 블루팀(0) 값 변경
- 3. GameLength : 분초를 초단위로 바꿔주기
- 4. ',' 단위의 문자열은 ','를 없애고 int 타입으로 변환
- 5.  그 외 값들을 모두 int 타입으로 변환

In [65]:
def Change_Data(x):
    try :
        if x == "승리" or x == "레드팀": return 1
        elif x == "패배" or x == "블루팀" : return 0
        elif x[-1] == "초":
            start = x.index("분 ")
            return (int(x[:start]) * 60) + int(x[start + 2:-1])
        elif ',' in x:
            x = x.replace(',','')
            return int(x)
        else: 
            return int(x)
    except:
        return x

### 13.2 Apply를 통해 Change_Data 함수 적용
- 모든 열에 Change_Data 함수 적용
- 새로운 데이터 프레임을 만들어서 저장

In [67]:
OPGG_chdf = pd.DataFrame()

for i in OPGG_df.columns:
    OPGG_chdf[i] = OPGG_df[i].apply(Change_Data)

In [68]:
OPGG_chdf

,GameResult,Team,GameLength,Win_Champion_kill,Lose_Champion_kill,Win_Money,Lose_Money,Win_in_Attack_num,Lose_in_Attack_num,Win_out_Attack_num,Lose_out_Attack_num,Win_CS_Count,Lose_CS_Count
0,1,1,1157,51,51,73527,69816,100149,120379,132125,108126,221,209
1,0,0,1218,25,21,63076,60438,100951,67485,71404,105481,239,220
2,0,1,545,34,22,36879,33504,51300,42176,50545,56216,99,73
3,0,1,1138,53,51,68069,66054,144150,138472,155341,151370,209,206
4,1,0,974,31,23,55895,51306,70761,69983,74060,75905,197,192
...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,0,1,976,46,35,60715,54400,98304,93747,107576,104652,195,164
93,1,0,1162,42,46,66111,64496,103216,112600,122896,107625,235,235
94,0,1,1189,45,44,68619,67188,119711,111846,119952,124372,234,245
95,1,0,1210,39,42,67918,65295,89534,118738,127415,91888,250,235


## Step 14. CSV 파일로 저장하기
- 원본 데이터와 변환된 데이터를 CSV 파일로 저장하기
- 인덱스 Number는 가져오지 않음

In [69]:
OPGG_chdf.to_csv(dir_name + "\\" + save_name + "_Change.csv", encoding="utf-8-sig", index=None)
OPGG_df.to_csv(dir_name + "\\" + save_name + "_Base.csv", encoding="utf-8-sig", index=None)

# 번외. 수집 데이터 합치기

In [4]:
print(os.getcwd())
 
_dir = os.listdir(os.getcwd() + '/OPGG_Crawling_save_data/0. ALL_Data')
print(_dir)

C:\AI\OPGG_DataCrawling
['154kv덕림cnt(20)_Change.csv', 'U 4cnt(100)_Change.csv', 'Zoacnt(100)_Change.csv', '교미크로스cnt(20)_Change.csv', '라이얻견제받는중cnt(20)_Change.csv', '로니팡cnt(500)_Change.csv', '바삭하게튀긴치킨cnt(100)_Change.csv', '백골부대특전사cnt(100)_Change.csv', '아아땡긴다cnt(20)_Change.csv', '으흐이cnt(100)_Change.csv', '주름백과사전cnt(100)_Change.csv']


In [5]:
result = pd.DataFrame()
for i in _dir:
    df1 = pd.read_csv('OPGG_Crawling_save_data/0. ALL_Data/' + i)
    result = pd.concat([result,df1]) 
result

,GameResult,Team,GameLength,Win_Champion_kill,Lose_Champion_kill,Win_Money,Lose_Money,Win_in_Attack_num,Lose_in_Attack_num,Win_out_Attack_num,Lose_out_Attack_num,Win_CS_Count,Lose_CS_Count
0,1,0,918,33,32,54073,49344,90254,57335,66670,99948,163,124
1,1,1,880,33,21,51246,45988,63101,42448,46627,65556,175,135
2,0,0,981,36,22,57330,51162,74630,61443,65847,77164,191,180
3,1,0,1507,47,46,82714,80449,115282,135324,151809,121278,345,313
4,0,1,1195,58,59,71700,69366,123539,114443,119753,130642,238,222
...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,1,0,828,39,31,51286,47292,79090,71701,78902,81809,159,164
94,0,1,1283,52,44,75788,70639,154900,145099,158136,168787,257,214
95,1,0,1017,28,28,56918,55828,93002,82715,89626,100823,203,216
96,0,0,1444,56,60,83364,80644,153849,206851,220048,170456,299,290


In [10]:
result.to_csv(dir_name + "\\" + "OPGG_Result_Data.csv", encoding="utf-8-sig", index=None)

In [12]:
num = 412

print(type(str(num)[-1]))

<class 'str'>
